# How To: Use a combinations matrix to keep track of combinabilities

## Preparation

In [1]:
# Set up the path to SModelS installation folder
import sys; sys.path.append("."); import smodels_paths

In [2]:
# Import those parts of smodels that are needed for this exercise
from smodels.tools.physicsUnits import GeV
from smodels.experiment.databaseObj import Database

In [3]:
# a combinations matrix is just a dictionary with combinable analyses as both keys and values
combinationsmatrix = { "CMS-SUS-21-002": [ "CMS-SUS-19-006"]}

In [4]:
## Load the official database:
db = Database("official", combinationsmatrix=combinationsmatrix)

In [20]:
# get a few results
cms19006 = db.getExpResults(analysisIDs=["CMS-SUS-19-006"],dataTypes='upperLimit')[0]
cms21002 = db.getExpResults(analysisIDs=["CMS-SUS-21-002"],dataTypes='upperLimit')[0]
cms20004 = db.getExpResults(analysisIDs=["CMS-SUS-20-004"],dataTypes='efficiencyMap')[0]
cms13012 = db.getExpResults(analysisIDs=["CMS-SUS-13-012"],dataTypes='upperLimit')[0]
atlas1841 = db.getExpResults(analysisIDs=["ATLAS-SUSY-2018-41"],dataTypes='upperLimit')[0]

## Check for combinability

In [ ]:
# we marked CMS-SUS-21-002 to be combinable with CMS-SUS-19-006 ...
cms19006.isCombinableWith ( cms21002 )

In [25]:
# combinability is Abelian
cms21002.isCombinableWith ( cms19006 )

True

In [26]:
# we did not define CMS-SUS-20-004
cms19006.isCombinableWith ( cms20004 )

False

In [27]:
# CMS analyses are always considered to be combinable with ATLAS
cms19006.isCombinableWith ( atlas1841 )

True

In [28]:
# results at different sqrt(s) are always considered to be combinable
cms19006.isCombinableWith ( cms13012 )

True

In [30]:
# an analysis cannot be combined with itself
cms19006.isCombinableWith ( cms19006 )

False